In [4]:
import pandas as pd

path = "/Users/gun/Desktop/미래에셋 AI 공모전/data/sector_info.csv"
df = pd.read_csv(path, dtype={"stock_code": str})

# '확인불가' 또는 '기타 산업군'인 종목 확인
mask = df['sector'].isin(['확인불가', '기타산업군'])
missing = df[mask]
print(missing[['stock_code', 'sector']])


     stock_code sector
5        009540   확인불가
22       013570   확인불가
29       107590   확인불가
38       055550  기타산업군
43       009970  기타산업군
...         ...    ...
1640     078070   확인불가
1642     900110   확인불가
1654     900310   확인불가
1674     003380   확인불가
1675     023760   확인불가

[91 rows x 2 columns]


In [ ]:
import pandas as pd
import requests
import io
import urllib3
from pathlib import Path

# SSL 경고 무시
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# 1) 경로 설정 (Pathlib 활용)
base_dir    = Path("/Users/gun/Desktop/미래에셋 AI 공모전/data")
input_path  = base_dir / "sector_info.csv"
output_path = base_dir / "sector_info_filled.csv"

# 2) 기존 CSV 불러오기
df = pd.read_csv(input_path, dtype={"stock_code": str})

# 3) '확인불가' 및 '기타 산업군' 마스크
mask = df['sector'].isin(['확인불가', '기타 산업군'])

# 4) KRX 공식 엑셀 다운로드 URL
url_kospi  = "https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13"
url_kosdaq = "https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=12"

# 5) Insecure HTTP 요청
resp_kospi  = requests.get(url_kospi,  verify=False)
resp_kosdaq = requests.get(url_kosdaq, verify=False)

# 6) BytesIO → DataFrame
df_kospi  = pd.read_excel(io.BytesIO(resp_kospi.content),  dtype={'종목코드': str})
df_kosdaq = pd.read_excel(io.BytesIO(resp_kosdaq.content), dtype={'종목코드': str})

# 7) 합치기 & 코드 포맷 6자리
df_all = pd.concat([df_kospi, df_kosdaq], ignore_index=True)
df_all['종목코드'] = df_all['종목코드'].str.zfill(6)

# 8) 매핑 딕셔너리 (종목코드 → 업종)
sector_map = dict(zip(df_all['종목코드'], df_all['업종']))

# 9) 결측 구간에 매핑 적용
df.loc[mask, 'sector'] = df.loc[mask, 'stock_code'].map(sector_map)

# 10) 결과 저장
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"✅ 완료: '{output_path}'에 저장되었습니다.")

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)>